In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from torch.optim import Adam
from peft import get_peft_model, LoraConfig, TaskType
from task2_signal import generate_signal
from task2_signal import generate_volatility_signal
from task2_stocks import get_stock_data
from task2_news import get_news


# Date ranges for the starter solution
END_DATE = "2023-10-16"
START_DATE = "2022-12-16"

"""a very simple env whost state space is only the data"""
STOCK_TICKERS_HIGHEST_CAP_US = [
    "AAPL"
]

"""load data and make dset - first we load in the ticker data for each ticker, then we enrich that with news data"""

stock_data = pd.read_csv("./competition_dsets/task2_stocks.csv")


stock_data['Date'] = pd.to_datetime(stock_data['Date'])


stock_data = stock_data[(stock_data['Ticker'].isin(STOCK_TICKERS_HIGHEST_CAP_US)) &
                        (stock_data['Date'] >= START_DATE) &
                        (stock_data['Date'] <= END_DATE)]

stock_data.head()

/nfs/home/klt0112/.conda/envs/financerag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-07 19:40:58.774795: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-07 19:40:58.792718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731008458.812425  761731 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731008458.818450  761731 cuda_bla

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
746,2022-12-16,136.690002,137.649994,133.729996,134.509995,133.256653,160156900,AAPL
747,2022-12-19,135.110001,135.199997,131.320007,132.369995,131.136597,79592600,AAPL
748,2022-12-20,131.389999,133.250000,129.889999,132.300003,131.067261,77432800,AAPL
749,2022-12-21,132.979996,136.809998,132.750000,135.449997,134.187897,85928000,AAPL
750,2022-12-22,134.350006,134.559998,130.300003,132.229996,130.997910,77852100,AAPL


In [2]:
# Load ESG data
esg_data = pd.read_csv("esg_data.csv")
esg_data['Date'] = pd.to_datetime(esg_data['Date'])

technical_data = pd.read_csv("AAPL_predictions_cleaned.csv")
technical_data['Date'] = pd.to_datetime(technical_data['Date'])

In [3]:
# Sort by date to ensure correct chronological order
esg_data = esg_data.sort_values('Date').reset_index(drop=True)
# Calculate the 1-month percentage difference
esg_data['ESG_pct_diff_1m'] = esg_data['ESG_score'].pct_change(periods=30) * 100  
esg_data['ESG_score'] = esg_data['ESG_pct_diff_1m']


In [4]:
from task2_env import Task2Env

bnb_config_4 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # or torch.bfloat16
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="fp4",  # 'nf4' or 'fp4'
)

bnb_config_8 = BitsAndBytesConfig(load_in_8bit=True)

num_gpus = torch.cuda.device_count()
max_memory = {i: "16GiB" for i in range(num_gpus)}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # map to auto for multi gpu
model_name = "meta-llama/Llama-3.2-3B-Instruct"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # quantization_config=bnb_config_4,
    device_map="auto",
    # max_memory=max_memory,
)
# model = torch.nn.DataParallel(model)
# model.to(device)

# grad checkpntg
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.config.use_cache = False

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.62s/it]


In [6]:
# LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=30,
    target_modules=["q_proj", "v_proj"],
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Initialize environment
from task2_env import Task2Env

# Assuming model, tokenizer, stock_data, esg_data, and tech_analysis_data are already defined
task2env = Task2Env(
    model=model,
    tokenizer=tokenizer,
    stock_data=stock_data,
    esg_data=esg_data,
    tech_analysis_data=technical_data,  # New parameter for technical analysis
    scale_range=(-10, 10),
    max_steps=252 - 4,
    threshold=3,
    lookahead = 3, 
    strong_positive_return=0.05,  # Strong positive return threshold
    moderate_positive_return=0.02,  # Moderate positive return threshold
    weak_positive_return=0.005,  # Weak positive return threshold
    low_risk_threshold=4,  # Updated low-risk volatility threshold
    reward_hyperparams={
        'great_positive_reward': 2.0,
        'moderate_positive_reward': 1.5,
        'weak_positive_reward': 1.0,
        'passive_reward': 0.0,
        'moderate_negative_reward': -1.0,
        'strong_negative_reward': -1.5,
        'esg_bonus': 0.1,
    }
)

state = task2env.reset()
actions = []
rewards = []
returns = []
running_eval = []
losses = []

optimizer = Adam(model.parameters(), lr=1e-5)

trainable params: 8,601,600 || all params: 3,221,351,424 || trainable%: 0.2670


In [8]:
# Define parameters for tracking and adjusting rewards
max_train_steps = 180
previous_eval_values = []  # List to store recent portfolio evaluations
trend_days = 3  # Number of consecutive days to consider for trend analysis
increase_factor = 1.1  # Factor to increase rewards if portfolio trend is positive
decrease_factor = 1.1  # Factor to increase negative penalties if portfolio trend is negative

for step in tqdm(range(max_train_steps), desc=f"Training for max steps: {max_train_steps}"):
    date, prices = state
    date = pd.Timestamp(date)
    done = False

    news = get_news(
            "AAPL",
            (date - timedelta(days=1))._date_repr,
            (date - timedelta(days=11))._date_repr,
            "./competition_dsets/task2_news.csv",
        )

    # Generate sentiment score and log probability
    sentiment_score, log_prob_sentiment = generate_signal(
        tokenizer,
        model,
        device,
        news,
    )

    # Prepare recent prices for volatility analysis
    recent_prices = stock_data[
        (stock_data['Date'] <= date.strftime('%Y-%m-%d')) &
        (stock_data['Date'] > (date - timedelta(days=5)).strftime('%Y-%m-%d'))
    ]['Close'].tolist()

    # Generate volatility score and log probability
    volatility_score, log_prob_volatility = generate_volatility_signal(
        tokenizer,
        model,
        device,
        news,
        recent_prices,
    )

    action = {
        'sentiment': sentiment_score,
        'volatility': volatility_score,
    }

    state, reward, done, info = task2env.step(action)
    actions.append(action)
    rewards.append(reward)
    returns.append(info["price change"])
    running_eval.append(info["running eval"])

    # Append current eval value and keep only the last 'trend_days' values
    previous_eval_values.append(info["running eval"])
    if len(previous_eval_values) > trend_days:
        previous_eval_values.pop(0)

    # Check if portfolio has increased or decreased consistently over 'trend_days'
    if len(previous_eval_values) == trend_days:
        # Check if all values in the trend are increasing
        if all(previous_eval_values[i] < previous_eval_values[i + 1] for i in range(trend_days - 1)):
            # Increase positive rewards if portfolio has been increasing for 'trend_days' consecutively
            task2env.reward_hyperparams['great_positive_reward'] *= increase_factor
            task2env.reward_hyperparams['moderate_positive_reward'] *= increase_factor
            task2env.reward_hyperparams['weak_positive_reward'] *= increase_factor
        elif all(previous_eval_values[i] > previous_eval_values[i + 1] for i in range(trend_days - 1)):
            # Increase negative penalties if portfolio has been decreasing for 'trend_days' consecutively
            task2env.reward_hyperparams['moderate_negative_reward'] *= decrease_factor
            task2env.reward_hyperparams['strong_negative_reward'] *= decrease_factor

        # Set hyperparameters for the advanced loss function
    alpha_sentiment = 0.7  # Weight for sentiment log probability
    alpha_volatility = 0.3  # Weight for volatility log probability
    risk_aversion = 0.5     # Penalizes risk to prevent extreme losses
    smoothing_factor = 0.1  # Penalizes sudden portfolio changes

    # Advanced loss calculation
    reward_tensor = torch.tensor(reward, device=device)

    # Weighted log probabilities for sentiment and volatility
    log_prob_combined = alpha_sentiment * log_prob_sentiment + alpha_volatility * log_prob_volatility

    # Risk-adjusted reward (e.g., Sharpe-inspired): reward - (risk_aversion * volatility_score)
    risk_adjusted_reward = reward_tensor - risk_aversion * torch.tensor(volatility_score, device=device)

    # Temporal consistency: penalize large changes in portfolio value (using `returns`)
    if len(returns) > 1:
        recent_return_change = returns[-1] - returns[-2]
        temporal_penalty = smoothing_factor * torch.abs(torch.tensor(recent_return_change, device=device))
    else:
        temporal_penalty = torch.tensor(0.0, device=device)

    # Define the advanced loss as the weighted negative log probability adjusted by risk and consistency
    loss = -(log_prob_combined * risk_adjusted_reward) + temporal_penalty
    loss = loss.mean()

    # Standard backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

    if done:
        break

Training for max steps: 180:   0%|          | 0/180 [00:00<?, ?it/s]

0.0


Training for max steps: 180:   1%|          | 1/180 [00:06<18:59,  6.36s/it]

9.0


Training for max steps: 180:   1%|          | 2/180 [00:12<17:54,  6.04s/it]

...

Training for max steps: 180:  99%|█████████▉| 178/180 [17:00<00:11,  5.82s/it]

10.0


Training for max steps: 180:  99%|█████████▉| 179/180 [17:06<00:05,  5.85s/it]

0


Training for max steps: 180: 100%|██████████| 180/180 [17:12<00:00,  5.74s/it]


In [9]:
# Plotting results
plt.figure(figsize=(16, 16))

# Plot Loss over Time
plt.subplot(3, 2, 1)
plt.plot(losses, label="Loss", color="red")
plt.title("Loss over Time")
plt.xlabel("Training Step")
plt.ylabel("Loss")
plt.legend()

# Plot Reward over Time
plt.subplot(3, 2, 2)
plt.plot(rewards, label="Reward", color="blue")
plt.title("Reward over Time")
plt.xlabel("Training Step")
plt.ylabel("Reward")
plt.legend()

# Plot Average Price Change over Time
plt.subplot(3, 2, 3)
plt.plot(returns, label="Average Price CHange", color="green")
plt.title("Average Price Change over Time")
plt.xlabel("Training Step")
plt.ylabel("Average Price Change")
plt.legend()

# Plot Running Evaluation Amount over Time
plt.subplot(3, 2, 4)
plt.plot(running_eval, label="Running Portfolio Amount", color="purple")
plt.title("Running Portfolio Amount over Time")
plt.xlabel("Training Step")
plt.ylabel("Portfolio Amount")
plt.legend()


# Extract sentiment and volatility values from the actions
sentiments = [action['sentiment'] for action in actions]
volatilities = [action['volatility'] for action in actions]
steps = list(range(1, len(actions) + 1))

# Plot Sentiment
plt.subplot(3, 2, 5)
plt.plot(steps, sentiments, marker='o', linestyle='-', label='Sentiment')
plt.xlabel('Step')
plt.ylabel('Sentiment Score')
plt.title('Sentiment over Time')
plt.legend()

# Plot Volatility
plt.subplot(3, 2, 6)
plt.plot(steps, volatilities, marker='o', linestyle='-', color='orange', label='Volatility')
plt.xlabel('Step')
plt.ylabel('Volatility Score')
plt.title('Volatility over Time')
plt.legend()

plt.tight_layout()
plt.savefig("training.png")